Topic: Challenge Set 1 Subject: Explore MTA turnstile data Date: 04/13/2018 Name: student name Worked with: other students' name Location: sea18_ds10/student_submissions/challenges/01-mta/shaikh_reshama/challenge_set_1_reshama.ipynb

In [20]:
import sys

In [21]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

import pprint

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

import dateutil.parser
from datetime import *

In [22]:
# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 3)

In [23]:
#read data from this folder
df = pd.read_csv('turnstile_171007.csv')
mta = df.copy()

In [24]:
#Work with turnstile name as combo

mta['TURNSTILE'] = mta['C/A'] + mta['UNIT'] + mta['SCP']
mta.drop('C/A', axis=1, inplace = True)
mta.drop('UNIT', axis=1,inplace = True)
mta.drop('SCP', axis=1, inplace = True)


In [25]:
#re-order cols

cols = mta.columns.tolist()
cols.insert(0, cols.pop(cols.index('TURNSTILE')))
mta = mta.reindex(columns= cols)


In [26]:
#drop useless data

mta = mta.drop('LINENAME', axis=1)
mta = mta.drop('DIVISION', axis=1)
mta = mta.drop('DESC', axis=1)

In [27]:
#remove exit as well

mta.columns = mta.columns.str.strip()

In [28]:
# make new columns, blank

leng = len(mta.index)

mta['ENT_COUNT'] = [0] * leng
mta['EXT_COUNT'] = [0] * leng
mta['DATE_TIME'] = [0] * leng
mta['Cu_ENT'] = [0] * leng

In [29]:
#reorder the date time column

cols = mta.columns.tolist()
cols.insert(4, cols.pop(cols.index('DATE_TIME')))
mta = mta.reindex(columns= cols)

In [30]:
# Fill empty date time column with formula
# I dont think we will use this...

for i in range(leng):
    datetime = df.DATE[i]+ ' ' + df.TIME[i]      
    value = dateutil.parser.parse(datetime)
     
    mta.at[i,'DATE_TIME'] = value

In [31]:
#fill empty ent_count with formula

for i in range(leng):
    if i == 0: 
        mta.at[i,'ENT_COUNT'] = 0
    else:
        if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
            diff = mta.ENTRIES[i] - mta.ENTRIES[i-1]
            # ignore any negative entry count or very large entry counts, turnstile malfunction
            # we should discuss a better 
            if diff < 0: diff = 0
            if diff > 10000: diff = 0
            mta.at[i,'ENT_COUNT'] = diff
        else:
            mta.at[i,'ENT_COUNT'] = 0

In [32]:
#fill in new exit counts
for i in range(leng):
    if i == 0: 
        mta.at[i,'EXT_COUNT'] = 0
    else:
        if mta.TURNSTILE[i] == mta.TURNSTILE[i-1]:
            diff = mta.EXITS[i] - mta.EXITS[i-1]
            # ignore any negative entry count or very large entry counts, turnstile malfunction
            # we should discuss a better 
            if diff < 0: diff = 0
            if diff > 10000: diff = 0
            mta.at[i,'EXT_COUNT'] = diff
        else:
            mta.at[i,'EXT_COUNT'] = 0

In [82]:
#mta = mta.drop('DATE', axis=1)
mta = mta.drop('TIME', axis=1)
mta = mta.drop('ENTRIES', axis=1)
mta = mta.drop('Cu_ENT', axis=1)
mta = mta.drop('EXITS', axis=1)

ValueError: labels ['TIME'] not contained in axis

In [34]:
mta.head()

,TURNSTILE,STATION,DATE,TIME,DATE_TIME,ENTRIES,EXITS,ENT_COUNT,EXT_COUNT,Cu_ENT
0,A002R05102-00-00,59 ST,09/30/2017,00:00:00,2017-09-30 00:00:00,6346858,2147624,0,0,0
1,A002R05102-00-00,59 ST,09/30/2017,04:00:00,2017-09-30 04:00:00,6346893,2147630,35,6,0
2,A002R05102-00-00,59 ST,09/30/2017,08:00:00,2017-09-30 08:00:00,6346912,2147655,19,25,0
3,A002R05102-00-00,59 ST,09/30/2017,12:00:00,2017-09-30 12:00:00,6347003,2147741,91,86,0
4,A002R05102-00-00,59 ST,09/30/2017,16:00:00,2017-09-30 16:00:00,6347270,2147801,267,60,0


In [35]:
# Check all negative and large numbers have been removed

mta.describe()

,ENTRIES,EXITS,ENT_COUNT,EXT_COUNT,Cu_ENT
count,1.967e+05,1.967e+05,196663.000,196663.000,196663.0
mean,4.003e+07,3.241e+07,181.638,138.153,0.0
std,2.080e+08,1.901e+08,265.700,233.081,0.0
min,0.000e+00,0.000e+00,0.000,0.000,0.0
25%,5.243e+05,2.643e+05,8.000,6.000,0.0
50%,2.707e+06,1.510e+06,73.000,51.000,0.0
75%,6.954e+06,4.840e+06,251.000,168.000,0.0
max,2.130e+09,2.097e+09,4241.000,4329.000,0.0


In [36]:
tur_uni = mta.TURNSTILE.unique()

In [37]:
"""# Find some time of day trend?
turn = ''
date = ''
sum_ent = 0
cu_list = []
for i in range(leng):
    #Initialing complex if, goal is to have the cumulative for each day 
    if turn != mta.TURNSTILE[i] and i==0:
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent= 0
    #if the turnstile and date are the same, sum up the ent_count
    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):
        sum_ent += mta['ENT_COUNT'][i]
    #if either the turnstile number changes, or the date changes, dump the sum of entries, 
    #and store in a new column across multpile indexes
    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:
        for j in range(i-beg_in):
            j +=beg_in
            mta.at[j, 'Cu_ENT'] = sum_ent 
        turn = mta.TURNSTILE[i]
        date = mta.DATE[i]
        beg_in = i
        sum_ent = mta['ENT_COUNT'][i]
mta"""

"# Find some time of day trend?\nturn = ''\ndate = ''\nsum_ent = 0\ncu_list = []\nfor i in range(leng):\n    #Initialing complex if, goal is to have the cumulative for each day \n    if turn != mta.TURNSTILE[i] and i==0:\n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent= 0\n    #if the turnstile and date are the same, sum up the ent_count\n    elif (turn == mta.TURNSTILE[i] and date == mta.DATE[i]):\n        sum_ent += mta['ENT_COUNT'][i]\n    #if either the turnstile number changes, or the date changes, dump the sum of entries, \n    #and store in a new column across multpile indexes\n    elif turn != mta.TURNSTILE[i] or date != df.DATE[i]:\n        for j in range(i-beg_in):\n            j +=beg_in\n            mta.at[j, 'Cu_ENT'] = sum_ent \n        turn = mta.TURNSTILE[i]\n        date = mta.DATE[i]\n        beg_in = i\n        sum_ent = mta['ENT_COUNT'][i]\nmta"

In [47]:
mt = mta.groupby(['STATION','DATE'])
ent_agg = mt.agg({'ENT_COUNT':['sum']})
ent_agg.reset_index(inplace=True)
ext_agg = mt.agg({'EXT_COUNT':['sum']})
#returns "index" as columns .reset_index()
ext_agg.reset_index(inplace=True)
ent_agg

,STATION,DATE,ENT_COUNT
,,,sum
0,1 AV,09/30/2017,12883
1,1 AV,10/01/2017,14815
2,1 AV,10/02/2017,20711
3,1 AV,10/03/2017,21932
4,1 AV,10/04/2017,22605
5,1 AV,10/05/2017,22814
6,1 AV,10/06/2017,23237
7,103 ST,09/30/2017,18169
8,103 ST,10/01/2017,16735


In [92]:
ent_agg.columns = ent_agg.columns.droplevel(level=1)
ext_agg.columns = ext_agg.columns.droplevel(level=1)

#Creating a new data frame removing repeted values
q_mta = pd.merge(ent_agg, ext_agg, on=['STATION','DATE'], how='left')



In [96]:
q_mta.sort_values(by=['ENT_COUNT'], ascending = 0, inplace = True)
uni_stations = q_mta.STATION.unique()[:30]
len(uni_stations)
print(uni_stations)

['34 ST-PENN STA' 'GRD CNTRL-42 ST' '23 ST' '34 ST-HERALD SQ'
 '14 ST-UNION SQ' 'TIMES SQ-42 ST' 'FULTON ST' '86 ST' '42 ST-PORT AUTH'
 '59 ST COLUMBUS' '125 ST' '59 ST' 'CHAMBERS ST' 'CANAL ST' '96 ST'
 '47-50 STS ROCK' '14 ST' 'FLUSHING-MAIN' '28 ST' '50 ST'
 'JKSN HT-ROOSVLT' 'JAY ST-METROTEC' 'WALL ST' 'ATL AV-BARCLAY' '72 ST'
 '42 ST-BRYANT PK' 'W 4 ST-WASH SQ' '7 AV' 'BOROUGH HALL' 'JAMAICA CENTER']


In [ ]:
# Next step! Combine days together and analyze what locations are the most popular and one what days
import seaborn as sns
g = sns.FacetGrid(mta, row='STATION', col='TURNSTILE', sharex=True, sharey=True)
g.map(plt.hist, "Cu_ENT")


#### Challenge 1   
  
- Open up a new IPython notebook
- Download a few MTA turnstile data files
- Open up a file, use csv reader to read it, make a python dict where
  there is a key for each (C/A, UNIT, SCP, STATION). These are the
  first four columns. The value for this key should be a list of
  lists. Each list in the list is the rest of the columns in a
  row. For example, one key-value pair should look like


{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'],
['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'],
['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'],
...
]
}

#### Challenge 2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}



#### Challenge 3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.


#### Challenge 4

We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the
notebook. To plot the time series, import matplotlib with

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the
turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)
    

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#### Challenge 5

- So far we've been operating on a single turnstile level, let's
  combine turnstiles in the same ControlArea/Unit/Station combo. There
  are some ControlArea/Unit/Station groups that have a single
  turnstile, but most have multiple turnstilea-- same value for the
  C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each
ControlArea/UNIT/STATION combo, for each day, add the counts from each
turnstile belonging to that combo.

#### Challenge 6

Similarly, combine everything in each station, and come up with a time
series of `[(date1, count1),(date2,count2),...]` type of time series
for each STATION, by adding up all the turnstiles in a station.

#### Challenge 7

Plot the time series for a station.

#### Challenge 8

- Make one list of counts for **one** week for one station. Monday's
count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another
week.
`plt.plot(week_count_list)` for every `week_count_list` you created
this way. You should get a rainbow plot of weekly commute numbers on
top of each other.

#### Challenge 9

- Over multiple weeks, sum total ridership for each station and sort
  them, so you can find out the stations with the highest traffic
  during the time you investigate

#### Challenge 10

- Make a single list of these total ridership values and plot it with

    plt.hist(total_ridership_counts)

to get an idea about the distribution of total ridership among
different stations.

This should show you that most stations have a small traffic, and the
histogram bins for large traffic volumes have small bars.

*Additional Hint*:

If you want to see which stations take the meat of the traffic, you
can sort the total ridership counts and make a `plt.bar` graph. For
this, you want to have two lists: the indices of each bar, and the
values. The indices can just be `0,1,2,3,...`, so you can do

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)